In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to SQLite (create a database in memory)
conn = sqlite3.connect(':memory:') 

# Load CSV files into pandas DataFrames
netflix_data = pd.read_csv('netflix_titles.csv')

# Load DataFrames into SQLite tables
netflix_data.to_sql('netflix_data', conn, index=False, if_exists='replace')

8807

In [3]:
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
netflix_data.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [5]:
netflix_data['director'].fillna('No info', inplace = True)
netflix_data['cast'].fillna('No info', inplace = True)
netflix_data['country'].fillna('No info', inplace = True)
netflix_data['date_added'].fillna('No info', inplace = True)
netflix_data['rating'].fillna('0', inplace = True)
netflix_data['duration'].fillna('0', inplace = True)

In [6]:
netflix_data.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

#  15 Business Problems & Solutions

1. Count the number of Movies vs TV Shows
2. Find the most common rating for movies and TV shows
3. List all movies released in a specific year (e.g., 2020)
4. Find the top 5 countries with the most content on Netflix
5. Identify the longest movie
6. Find all the movies/TV shows by director 'Rajiv Chilaka'!
7. List all TV shows with more than 5 seasons
8. Count the number of content items in each genre
9. Find each year and the average numbers of content release in India on netflix. 
    return top 5 year with highest avg content release!
10. Categorize the content based on the presence of the keywords 'kill' and 'violence' in 
    the description field. Label content containing these keywords as 'Bad' and all other 
    content as 'Good'. Count how many items fall into each category.

In [7]:
query = '''
    SELECT type, COUNT(*) AS count
    FROM netflix_data
    GROUP BY type ;
'''

result = pd.read_sql(query, conn)
print(result)

      type  count
0    Movie   6131
1  TV Show   2676


In [13]:
query = '''
    WITH RatingCounts AS (
    SELECT 
        type,
        rating,
        COUNT(*) AS rating_count
    FROM netflix_data
    GROUP BY type, rating
),
RankedRatings AS (
    SELECT 
        type,
        rating,
        rating_count,
        RANK() OVER (PARTITION BY type ORDER BY rating_count DESC) AS rank
    FROM RatingCounts
)
SELECT 
    type,
    rating AS most_frequent_rating
FROM RankedRatings
WHERE rank = 1;
    
    
'''

result = pd.read_sql(query, conn)
print(result)

      type most_frequent_rating
0    Movie                TV-MA
1  TV Show                TV-MA


In [17]:
query = '''
    SELECT title
    FROM netflix_data
    WHERE release_year = 2020 ;
'''

result = pd.read_sql(query, conn)
print(result)

                                                 title
0                                 Dick Johnson Is Dead
1    Europe's Most Dangerous Man: Otto Skorzeny in ...
2                                      Falsa identidad
3                                        Sex Education
4                              Tayo and Little Wizards
..                                                 ...
948                                   #cats_the_mewvie
949                 Norm of the North: Family Vacation
950                                        Straight Up
951                                        Super Wings
952          Surviving R. Kelly Part II: The Reckoning

[953 rows x 1 columns]


In [34]:
query = '''
    SELECT country, COUNT(*) AS count
FROM netflix_data
GROUP BY country
ORDER BY count DESC
LIMIT 5;

'''

result = pd.read_sql(query, conn)
print(result)

          country  count
0   United States   2818
1           India    972
2            None    831
3  United Kingdom    419
4           Japan    245


In [28]:
query = '''
    SELECT title, duration FROM netflix_data
    WHERE type = 'Movie'
    ORDER BY CAST(SUBSTRING_INDEX(duration, ' ', 1))
    
'''

result = pd.read_sql(query, conn)
print(result)

                        title duration
0  Black Mirror: Bandersnatch  312 min


In [41]:
query = '''
    SELECT *
FROM netflix_data
WHERE director = 'Rajiv Chilaka';

'''

result = pd.read_sql(query, conn)
print(result)

   show_id   type                                              title  \
0     s407  Movie                       Chhota Bheem - Neeli Pahaadi   
1     s408  Movie                              Chhota Bheem & Ganesh   
2     s409  Movie                 Chhota Bheem & Krishna: Mayanagari   
3     s410  Movie  Chhota Bheem & Krishna: Pataliputra- City of t...   
4     s411  Movie                 Chhota Bheem And The Broken Amulet   
5     s412  Movie             Chhota Bheem And The Crown of Valhalla   
6     s413  Movie               Chhota Bheem and the Incan Adventure   
7     s414  Movie                Chhota Bheem and The ShiNobi Secret   
8     s415  Movie                           Chhota Bheem Aur Hanuman   
9     s416  Movie                           Chhota Bheem aur Krishna   
10    s417  Movie                Chhota Bheem aur Krishna vs Zimbara   
11    s420  Movie                      Chhota Bheem: Bheem vs Aliens   
12    s421  Movie               Chhota Bheem: Dholakpur to Kathm

In [43]:
query = '''
    SELECT *
FROM netflix_data
WHERE type = 'TV Show' 
  AND CAST(substr(duration, 1, instr(duration, ' ') - 1) AS INTEGER) > 5;

'''

result = pd.read_sql(query, conn)
print(result)

   show_id     type                          title         director  \
0       s9  TV Show  The Great British Baking Show  Andy Devonshire   
1      s56  TV Show                      Nailed It             None   
2      s66  TV Show                   Numberblocks             None   
3      s68  TV Show              Saved by the Bell             None   
4      s83  TV Show                        Lucifer             None   
..     ...      ...                            ...              ...   
94   s8190  TV Show         The Andy Griffith Show             None   
95   s8379  TV Show                     The L Word             None   
96   s8443  TV Show              The Office (U.S.)             None   
97   s8558  TV Show                  The West Wing             None   
98   s8711  TV Show                          Weeds             None   

                                                 cast                country  \
0   Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...         Unite

In [48]:
query = '''
    WITH RECURSIVE split_genres(listed_in, genre) AS (
    SELECT
        listed_in,
        substr(listed_in, 1, instr(listed_in || ',', ',') - 1) AS genre
    FROM netflix_data
    UNION ALL
    SELECT
        substr(listed_in, instr(listed_in || ',', ',') + 1),
        substr(substr(listed_in, instr(listed_in || ',', ',') + 1), 1, 
        instr(substr(listed_in, instr(listed_in || ',', ',') + 1) || ',', ',') - 1)
    FROM split_genres
    WHERE listed_in != ''
)
SELECT
    genre,
    COUNT(*) AS total_content
FROM split_genres
WHERE genre != ''
GROUP BY genre
ORDER BY total_content DESC;


'''

result = pd.read_sql(query, conn)
print(result)

                        genre  total_content
0        International Movies           2624
1                      Dramas           1600
2                    Comedies           1210
3          Action & Adventure            859
4               Documentaries            829
..                        ...            ...
68            Romantic Movies              3
69  Spanish-Language TV Shows              2
70        TV Sci-Fi & Fantasy              1
71              Sports Movies              1
72               LGBTQ Movies              1

[73 rows x 2 columns]


In [50]:
query = '''
    SELECT release_year, AVG(content_count) AS avg_content
FROM (
    SELECT release_year, COUNT(*) AS content_count
    FROM netflix_data
    WHERE country = 'India'
    GROUP BY release_year
) AS yearly_data
GROUP BY release_year
ORDER BY avg_content DESC
LIMIT 5;

'''

result = pd.read_sql(query, conn)
print(result)

   release_year  avg_content
0          2017        101.0
1          2018         94.0
2          2019         87.0
3          2020         75.0
4          2016         73.0


In [55]:
query = '''
    SELECT 
    CASE 
        WHEN description LIKE '%kill%' OR description LIKE '%violence%' THEN 'Bad'
        ELSE 'Good'
    END AS content_rating,
    COUNT(*) AS count
FROM netflix_data
GROUP BY content_rating;

'''

result = pd.read_sql(query, conn)
print(result)

  content_rating  count
0            Bad    342
1           Good   8465
